In [ ]:
import requests
import pandas as pd
import json
import geopandas as gpd
import io
from shapely.geometry import shape
import urllib.request as request
import json
import datetime
import os
import xarray as xr
import numpy as np
import seaborn as sns
import folium
import folium.plugins as plugins
from folium.features import DivIcon
from metpy.calc import dewpoint_from_relative_humidity, altimeter_to_sea_level_pressure, equivalent_potential_temperature
from metpy.units import units
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import base64

# ========= DOWNLOAD METADATA =========================
url_meta = "https://dataset.api.hub.geosphere.at/v1/station/current/tawes-v1-10min/metadata"

source = requests.get(url_meta).json()
# print(source)

folder = 'C:\\Users\\sh16450\\OneDrive - University of Bristol\\Documents\\GitHub\\openweatherviz\\folium\\current_csv\\'

# Open the station info for the current data and merge
df = pd.read_csv('C:\\Users\\sh16450\\OneDrive - University of Bristol\\Documents\\GitHub\\openweatherviz\\folium\\current_csv\\'
                                + 'corrected_station_info_current_tawes.csv').rename(columns={'id':'station'})




station_id_list = [str(df['station'][i]) for i in range(len(df['station']))]
# This gives all the station ID's in the current file for the download string
string_id = ",".join(station_id_list)

# create the date string from the metadata file to create a date string
# to save the file
date_str = source['time'].replace(':','-').split('T')
time_replaced = date_str[1].replace('+','-')

date_str_comb = date_str[0] + '_' + time_replaced

folder = 'C:\\Users\\sh16450\\OneDrive - University of Bristol\\Documents\\GitHub\\openweatherviz\\folium\\current_csv\\'
save_str = folder + date_str_comb + '_tawes_current_10min.csv'
print(save_str)

# ======== DOWNLOAD THE DATA =========================
download_str = ("https://dataset.api.hub.geosphere.at/v1/station/current/tawes-v1-10min?parameters=TL,TP,TLMAX,TLMIN,TS,FF,FFX,DD,DDX,GLOW,P,RF,PRED,RR,RRM,SCHNEE,SO"+
                "&station_ids="+string_id +"&output_format=csv")
print(download_str)

with open(save_str, 'wb') as out_file:
    content = requests.get(download_str).content
    out_file.write(content)
    print("I just saved file: {}".format(save_str))


C:\Users\sh16450\OneDrive - University of Bristol\Documents\GitHub\openweatherviz\folium\current_csv\2024-11-04_21-20-00-00_tawes_current_10min.csv
https://dataset.api.hub.geosphere.at/v1/station/current/tawes-v1-10min?parameters=TL,TP,TLMAX,TLMIN,TS,FF,FFX,DD,DDX,GLOW,P,RF,PRED,RR,RRM,SCHNEE,SO&station_ids=11369,11266,11375,11157,11301,11019,11125,11254,11018,11395,11275,11062,11188,11356,11218,11046,11234,11372,11244,11256,11361,11358,11264,11057,11367,11079,11197,11141,11099,11304,11051,11101,11129,11175,11394,11077,11318,11066,11267,11268,11285,11206,11302,11118,11190,11002,11232,11298,11105,11227,11235,11155,11173,11222,11104,11068,11228,11249,11246,11048,11312,11110,11064,11108,11247,11273,11371,8989103,8989104,11290,11291,11240,11238,11359,11037,11198,11082,11378,11135,8989044,11309,11167,11390,11259,11384,11339,11385,11333,11315,11297,8989114,11115,11121,11320,11360,11024,11325,11200,11362,11216,11176,11349,8989084,11327,11279,11331,8989076,11391,11192,8989078,11131,11255,11007

In [ ]:
# station_parameters = ""
# for parameter in source['parameters']:
#     station_parameters += parameter["name"] + ","
# station_parameters = station_parameters.rstrip(station_parameters[-1])
# all_station_parameters = station_parameters

# # print(all_station_parameters)

# station_list = source["stations"]
# stations = {}
# for station in station_list:
#     stations[station["id"]] = tuple(
#        float((str(station[coord])))
#         for coord in ("lat", "lon")
#                     )
    
# df = pd.DataFrame(station_list)
# df.to_csv(folder + 'station_info_current_tawes.csv')

In [3]:
met_data = pd.read_csv(save_str)
station_data = pd.read_csv(folder + 'station_info_current_tawes.csv').rename(columns={'id':'station'})

merged_data = pd.merge(met_data, station_data, on='station')
merged_data['FFXKMH'] = (merged_data['FFX'] * 3.6).round(decimals=1)
merged_data['FFKMH'] = (merged_data['FF'] * 3.6).round(decimals=1)

thetae = equivalent_potential_temperature(merged_data['P'].values * units.hPa, merged_data['TL'].values * units.degC, merged_data['TP'].values * units.degC).to('degC')
merged_data['THETAE'] = thetae.magnitude.round(decimals=1)

merged_data

,time,station,TL,TP,TLMAX,TLMIN,TS,FF,FFX,DD,...,lon,altitude,valid_from,valid_to,has_sunshine,has_global_radiation,is_active,FFXKMH,FFKMH,THETAE
0,2024-11-01T21:20+00:00,11001,9.8,7.5,9.8,9.6,7.3,3.1,3.9,281.0,...,13.670833,635.0,1972-01-01T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,14.0,11.2,33.2
1,2024-11-01T21:20+00:00,11002,6.5,6.5,7.2,6.5,5.5,1.0,1.9,312.0,...,14.450278,317.0,2008-04-21T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,6.8,3.6,24.5
2,2024-11-01T21:20+00:00,11004,4.4,4.0,4.4,4.3,2.1,0.8,1.3,195.0,...,13.376111,351.0,2007-12-05T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,4.7,2.9,20.1
3,2024-11-01T21:20+00:00,11007,7.1,6.8,7.2,7.1,7.1,2.1,3.0,240.0,...,13.839167,714.0,2007-12-05T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,10.8,7.6,30.5
4,2024-11-01T21:20+00:00,11008,6.1,5.5,6.7,5.9,3.4,0.3,0.8,120.0,...,14.004444,613.0,1972-01-01T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,2.9,1.1,26.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,2024-11-01T21:20+00:00,8989104,NaN,NaN,NaN,NaN,NaN,1.3,2.4,330.0,...,15.429444,351.0,2022-11-14T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,8.6,4.7,NaN
266,2024-11-01T21:20+00:00,8989106,-1.8,-1.8,-1.3,-1.8,-4.4,NaN,NaN,NaN,...,14.732778,845.0,2023-03-30T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,NaN,NaN,NaN
267,2024-11-01T21:20+00:00,8989113,1.1,0.9,1.1,0.9,NaN,0.2,0.5,22.0,...,14.852500,620.0,2023-10-16T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,1.8,0.7,NaN
268,2024-11-01T21:20+00:00,8989114,9.4,6.8,9.6,8.8,NaN,1.5,2.5,153.0,...,16.451944,312.0,2024-03-04T00:00+00:00,2025-11-01T21:30+00:00,False,False,True,9.0,5.4,NaN


In [12]:
source['time'].split('+')[0] + ' UTC'

'2024-10-31T21:00:00 UTC'

In [4]:

m = folium.Map(location=[47.73, 13.48], zoom_start=8, tiles=None)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=MglwGpuT1VUaVc2LSRX042X3krQz6y2cOZfGtNqWg3VMUke6gemUIWz7G1uxPPXP',
                 attr='CartoDB.Voyager', max_zoom=20).add_to(base_map)
base_map.add_to(m)

# Add the date as a title to the map
map_title = source['time'].split('+')[0] + ' UTC'
title_html = f'<h1 style="position:absolute;z-index:100000;left:10vw;background-color:#eeeee4;" >{map_title}</h1>'
m.get_root().html.add_child(folium.Element(title_html))


COLORS = [
    (lambda x: x > 40, "#EC8989"),
    (lambda x: 35 < x <= 40, "#EA3D09"),
    (lambda x: 30 < x <= 35, "#EA3D09"),
    (lambda x: 25 < x <= 30, "#EA6509"),
    (lambda x: 20 < x <= 25, "#C5C71B"),
    (lambda x: 15 < x <= 20, "#8EF700"),
    (lambda x: 10 < x <= 15, "#07AF02"),
    (lambda x: 5 < x <= 10, "#94EA7F"),
    (lambda x: 0 < x <= 5, "#9cecc8"),
    (lambda x: 0 >= x >= -5, "#18B5DA"),
    (lambda x: -5 > x >= -10, "#1661e8"),
    (lambda x: -10 > x >= -15, "#A375E6"),
    (lambda x: -15 > x >= -20, "#ce8bf0"),
    (lambda x: x < -20, "#EC9CE6"),
]

COLORS_WIND = [
    (lambda x: x >= 100, "#EF06E1"), # purple pizazz
    (lambda x: 80 <= x < 100, "#EF0620"), # torch red
    (lambda x: 60 <= x < 80, "#FB5D04"), # blaze orange
    (lambda x: 40 <= x < 60, "#D8DC0F"), # bitter lemon
    (lambda x: 30 <= x < 40, "#4FBF2A"), # apple
    (lambda x: x < 30, "#000000"), # black
    
    
]

COLORS_PREC_SUM = [
    (lambda x: x >= 80, "#EF06E1"), # purple pizazz
    (lambda x: 50 <= x < 80, "#EF0620"), # torch red
    (lambda x: 30 <= x < 50, "#FB5D04"), # blaze orange
    (lambda x: 10 <= x < 30, "#D8DC0F"), # bitter lemon
    (lambda x: 1 <= x < 10, "#4FBF2A"), # apple
    (lambda x: x < 1, "#000000"), # black
    
    
]




# This picks the color for temperature and tmax, dewpoint and wind
def get_color(x, color_list):
    for predicate, color in color_list:
        if predicate(x):
            return color

    return 'black'

# Add analysis for temperature
fg = folium.FeatureGroup(name="Temperature (last 10 min)", control=True, overlay=False).add_to(m)
# marker_cluster = plugins.MarkerCluster(options={'disableClusteringAtZoom':10, 'maxClusterRadius':5, 'singleMarkerMode':False}).add_to(m)

for i in range(np.shape(merged_data)[0]):
        color = get_color(merged_data['TL'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(merged_data['TL'].iloc[i]) + '</div>'
        p1 = [merged_data['lat'].iloc[i], merged_data['lon'].iloc[i]]
        folium.Marker(p1,
                      popup=merged_data['name'][i],
                      icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fg)

        # fg.add_child(popup_list[i])
        
# Add analysis for dewpoint temperature
fgg = folium.FeatureGroup(name="Dewpoint (last 10 min)", control=True, overlay=False, show=False).add_to(m)
# marker_cluster = plugins.MarkerCluster(options={'disableClusteringAtZoom':10, 'maxClusterRadius':5, 'singleMarkerMode':False}).add_to(m)

for i in range(np.shape(merged_data)[0]):
        color = get_color(merged_data['TP'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(merged_data['TP'].iloc[i]) + '</div>'
        p1 = [merged_data['lat'].iloc[i], merged_data['lon'].iloc[i]]
        folium.Marker(p1,
                      popup=merged_data['name'][i],
                      icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgg)
        
# Add analysis for maximum temperature        
fgg = folium.FeatureGroup(name="Tmax (last 10 min)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(merged_data)[0]):
        color = get_color(merged_data['TLMAX'].iloc[i], COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(merged_data['TLMAX'].iloc[i]) + '</div>'
        p1 = [merged_data['lat'].iloc[i], merged_data['lon'].iloc[i]]
        folium.Marker(p1,
                      popup=merged_data['name'][i],
                      icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fgg)
        
        
# Add analysis for ThetaE        
fggg = folium.FeatureGroup(name="Eq. potential Temperature (last 10 min)", control=True, overlay=False, show=False).add_to(m)

for i in range(np.shape(merged_data)[0]):
        # use temperature color but reduce by 20C
        color = get_color(merged_data['THETAE'].iloc[i] - 20, COLORS)
        html_str = '<div style="font-size: 14pt; color : ' + color +'">' + str(merged_data['THETAE'].iloc[i]) + '</div>'
        p1 = [merged_data['lat'].iloc[i], merged_data['lon'].iloc[i]]
        folium.Marker(p1,popup=merged_data['name'].iloc[i], icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(7,20),
                html= html_str,
                )).add_to(fggg)

        
folium.LayerControl().add_to(m)

plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


# icon = folium.DivIcon(html="17.4")
# folium.Marker([47.0, 15.0], icon=icon).add_to(m)
m.save(os.path.join('tawes_current.html'))

In [33]:
content

b'time,station,TL,TP,TLMAX,TLMIN,TS,FF,FFX,DD,DDX,GLOW,P,RF,PRED,RR,RRM,SCHNEE,SO\n2024-10-26T16:50+00:00,11001,9.8,9.8,9.8,9.8,10.1,5.5,8.9,98.0,84.0,0.0,947.1,100.0,1019.3,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11002,12.2,10.2,12.2,12.2,12.1,7.6,9.8,109.0,115.0,0.0,982.3,88.0,1020.0,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11004,12.7,10.7,12.7,12.7,11.6,3.5,6.0,92.0,103.0,0.0,977.1,88.0,1018.8,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11007,9.6,8.4,9.6,9.4,8.2,2.8,5.4,71.0,87.0,0.0,936.5,92.0,1019.7,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11008,10.6,8.7,10.6,10.5,8.2,3.4,5.6,36.0,37.0,0.0,947.9,89.0,1019.9,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11010,,,,,,,,,,,,,,,,,\n2024-10-26T16:50+00:00,11012,11.9,10.2,11.9,11.9,11.4,6.8,9.4,91.0,96.0,0.0,973.5,90.0,1019.7,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11018,12.6,10.4,12.6,12.6,12.2,2.9,5.8,77.0,63.0,0.0,989.0,87.0,1020.5,0.0,0.0,,0.0\n2024-10-26T16:50+00:00,11019,9.7,9.7,9.8,9.7,9.7,3.9,6.5,106.0,106.0,0.0,950.2,100.0,1021.6,0.0,3.0,0.0,0.0\n2024-10-26T1